<a href="https://colab.research.google.com/github/leandro-driguez/Machine-Learning-Techniques/blob/dev/Taller_2/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/hectormelo/Machine-Learning-Techniques/main/Banner.png" ><br>
# Machine Learning Techniques - MISIS4219

Primer semestre - 2024



# **Taller 2**

GeoAlpes, una empresa líder en análisis geoespacial, está buscando mejorar sus técnicas de clasificación automática de imágenes satelitales. El objetivo es poder categorizar distintas características geográficas (como bosques, zonas industriales, zonas de cultivos permanentes, zonas residenciales y rios) con alta precisión y rapidez. Para lograrlo, están interesados en explorar las capacidades de los métodos ensemble.

1.  Exploración y Preparación de Datos

    -   Discuta las particularidades de las imágenes satelitales y sugiera técnicas extra de preprocesamiento de ser necesario.

2.  Implementación de Gradient Boosting

    -   Utilice Gradient Boosting como modelo base. Discuta las ventajas y desafíos de este método, y cómo afecta la precisión y robustez del clasificador. Además, compare el desempeño de Gradient Boosting con el modelo Random Forest presentado en la práctica, ¿logra observar mejoras significativas?

3. Implementación de un nuevo metodo Ensemble
    - Elija y presente un método ensemble de su preferencia.Introduzca y discuta el concepto del método elegido y cómo podría ser benéfico para la clasificación de imágenes satelitales.
    - Compare el desempeño de su método elegido con Gradient Boosting y el modelo Random Forest. Discuta las ventajas y desventajas de cada uno.

4.  Optimización y Ajuste

    -   Realice una búsqueda de los mejores hiperparámetros para mejorar el desempeño de cada uno de los modelos implementados (Grid Search).

Datos: [Enlace al sub conjunto del dataset de imágenes satelitales EuroSAT](https://github.com/hectormelo/Machine-Learning-Techniques/raw/main/Lab_2/EuroSAT3.zip)


In [ ]:
import os
import numpy as np
from PIL import Image
from scipy import stats
from numpy import fliplr
from skimage.color import rgb2gray
from skimage.color import label2rgb
from skimage.transform import rotate
from skimage.segmentation import slic
from sklearn.model_selection import train_test_split

import pywt
import mahotas
import mahotas.features
from skimage.feature import local_binary_pattern

from sklearn.decomposition import PCA 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

import xgboost as xgb

In [ ]:
def load_and_preprocess_images(folder, img_size=(64, 64), use_slic_segmentation=False, n_segments=30, compactness=10):
    images = []
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        image = Image.open(image_path)
        image = np.array(image.convert("RGB"))
        
        # normalization
        normalized_image = image / 255.0
        
        # data augmentation
        rotated_image = rotate(normalized_image, angle=90)
        flipped_image = fliplr(normalized_image)
        
        # apply SLIC
        if use_slic_segmentation:
            segments = slic(normalized_image, n_segments=n_segments, compactness=compactness, start_label=1)
            segmented_image = label2rgb(segments, normalized_image, kind='avg', bg_label=0)
            images.append(segmented_image)
        else:
            images.extend([normalized_image, rotated_image, flipped_image])
    
    return images

In [ ]:
categories = ['Forest', 'Industrial', 'PermanentCrop', 'Residential', 'River']
base_path = './EuroSAT3/'

all_images = []
all_labels = []

for label, category in enumerate(categories):
    folder = os.path.join(base_path, category)
    category_images = load_and_preprocess_images(folder)
    all_images.extend(category_images)
    all_labels.extend([label] * len(category_images))

# Convertir listas a arrays de NumPy
all_images_np = np.array(all_images)
all_labels_np = np.array(all_labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_images_np, all_labels_np, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
def extract_color_stats(image):
    features = []

    for channel in range(image.shape[-1]):
        channel_data = image[:, :, channel].ravel()
        
        mean = np.mean(channel_data)
        std = np.std(channel_data)
        var = np.var(channel_data)
        median = np.median(channel_data)
        
        mode_result = stats.mode(channel_data)
        mode = mode_result.mode
        
        min_val = np.min(channel_data)
        max_val = np.max(channel_data)
        percentile_25 = np.percentile(channel_data, 25)
        percentile_50 = np.percentile(channel_data, 50)
        percentile_75 = np.percentile(channel_data, 75)
        skewness = stats.skew(channel_data)
        kurtosis = stats.kurtosis(channel_data)
        data_range = max_val - min_val
        
        features.extend([
            mean, std, var, median, mode, min_val, max_val,
            percentile_25, percentile_50, percentile_75,
            skewness, kurtosis, data_range
        ])

    return features


def extract_color_histogram(image, bins=32, channel_range=(0, 256)):
    histogram = [np.histogram(image[:, :, i], bins=bins, range=channel_range)[0] for i in range(image.shape[-1])]
    return np.concatenate(histogram)


def extract_haralick_features(image):
    gray_image = rgb2gray(image)
    gray_image = (gray_image * 255).astype('uint8')

    features = mahotas.features.haralick(gray_image).mean(axis=0)
    
    return features


def extract_lbp_features(image, P=8, R=1, method='uniform'):

    image_gray = rgb2gray(image)
    lbp = local_binary_pattern(image_gray, P=P, R=R, method=method)

    lbp_hist, _ = np.histogram(lbp, density=True, bins=np.arange(0, P + 3), range=(0, P + 2))
    return lbp_hist


def extract_fft_features(image):

    fft_coefs = np.fft.fft2(rgb2gray(image))
    fft_abs = np.abs(fft_coefs)

    return np.mean(fft_abs), np.std(fft_abs)


def extract_wavelet_features(image, mode='haar', level=1):
    coeffs2 = pywt.dwt2(rgb2gray(image), mode)
    cA, (cH, cV, cD) = coeffs2

    return np.mean(cA), np.mean(cH), np.mean(cV), np.mean(cD)


In [ ]:
def preprocess_image(image):
    color_stats_features = extract_color_stats(image)
    color_histogram_features = extract_color_histogram(image)
    haralick_features = extract_haralick_features(image)
    lbp_features = extract_lbp_features(image)
    fft_features = extract_fft_features(image)
    wlt_features = extract_wavelet_features(image)
    
    features = np.concatenate([
        color_stats_features, color_histogram_features, 
        haralick_features, lbp_features,
        fft_features, wlt_features
    ])
    return features

In [ ]:
X_train_preprocessed = np.array([preprocess_image(image) for image in X_train])
X_test_preprocessed = np.array([preprocess_image(image) for image in X_test])

In [ ]:
X_train_preprocessed.shape

In [ ]:
# Definir el modelo
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Definir la rejilla de hiperparámetros a buscar
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'subsample': [0.6, 0.8, 1.0],
    'tree_method': ['auto']
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Ejecutar la búsqueda
grid_search.fit(X_train_preprocessed, y_train)

# Mejores parámetros y mejor puntuación
print("Mejores parámetros encontrados: ", grid_search.best_params_)
print("Mejor puntuación: ", grid_search.best_score_)

In [ ]:
grid_search.best_params_

In [ ]:
y_pred = grid_search.predict(X_test_preprocessed)

In [ ]:
# Precisión
print("Precisión de Gradient Boosting:", accuracy_score(y_test, y_pred))

# Informe de clasificación
print("\nInforme de clasificación de Gradient Boosting:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de confusión de Gradient Boosting:")
cm_rf = confusion_matrix(y_test, y_pred)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf)
disp_rf.plot()